In [1]:

import os
import unittest
import pytest
import sqlite3

import pandas as pd

from pyfvs import fvs
from pyfvs.keywords import keywords as kwds

root = os.getcwd()

workspace = f'{root}/fia_test'
fia_db = f'{root}/fia_test.db'
fvs_variant = 'PN'
forest_code = 612

In [2]:
try:
    os.makedirs(workspace)
except:
    pass

with sqlite3.connect(fia_db) as conn:
    plot = pd.read_sql('select * from plot', conn).iloc[0]
    plot_cn = plot['cn']

    ## FIXME: Aggregate condition or constrain trees to condition
    cond = pd.read_sql(f"select * from cond where plt_cn='{plot_cn}'", conn).iloc[0]
    
    tree = pd.read_sql(f"select * from tree where plt_cn='{plot_cn}'", conn)
    # site_tree = pd.read_sql(f"select * from sitetree where plt_cn='{plot_cn}'", conn)

m = tree['statuscd']==1
live_trees = tree.loc[m,:].copy()

sim = fvs.FVS(fvs_variant, workspace=workspace, cleanup=False)
kw = sim.init_keywords(title=plot_cn)
kw += kwds.STANDCN(plot_cn)
kw += kwds.STDINFO(
    location=forest_code
    , slope=cond['slope']
    , aspect=cond['aspect']
    , elevation=plot['elev']
)
kw += kwds.DESIGN()
kw += kwds.TIMEINT(0,10)
kw += kwds.NUMCYCLE(1)

kw += kwds.SITECODE(int(cond['sisp']), cond['sicond'])
kw += kwds.NOTREES()

live_trees['plot_id'] = 1
live_trees['tree_num'] = live_trees['tree'].astype(int)
live_trees['tree_count'] = live_trees['tpa_unadj']
live_trees['species'] = live_trees['spcd'].astype(int)
live_trees['diam_growth'] = 0.0
live_trees['height_growth'] = 0.0
live_trees['total_height'] = 0.0
live_trees['age'] = 0.0

# cols = ['plot_id','tree_num','tree_count','species','diam']
# sim.inv_trees = live_trees.loc[:,cols]

# sim.init_projection(kw)
# r = sim.run()

# print('FVS Return Code: %s' % r)
# self.assertEqual(r, 0, 'FVS Return Code: %s' % r)


In [3]:
# sim.initialize_trees

In [4]:
cols = ['plot_id','tree_num', 'tpa_unadj', 'statuscd', 'species'
        , 'dia', 'diam_growth', 'ht', 'total_height'
        , 'height_growth', 'cr'
        , 'age']
x = live_trees.loc[:,cols]
args = [live_trees[c].values for c in cols]
# r = sim.initialize_trees(num_trees=live_trees.shape[0], *args)

In [5]:
inv = sim.fvslib.inventory_trees
inv.plot_id = x['plot_id'].values
inv.tree_id = x['tree_num'].values
inv.trees = x['tpa_unadj'].values
inv.tree_status = x['statuscd'].values
inv.diameter = x['dia'].values
inv.species = x['species'].values
# inv.plot_id = x['plot_id'].values

In [6]:
inv.print_trees()
# inv.species[0]

In [7]:
sim.init_projection(kw)
r = sim.initialize_trees(num_trees=live_trees.shape[0], *args)
r = sim.run()
print(r)

AttributeError: No FVS object "initialize_trees".

In [ ]:
sim.globals.prob[:10]

array([6.018046, 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      ], dtype=float32)